# CV 교차검증 스태팅 모델 소개******

## 스태킹(Stacking)은 개별 알고리즘의 예측 결과 데이터 세트를<br>                    최종적인 메타 데이터 세트로 만들어 별도의 ML알고리즘으로 최종 학습을 수행하는 방식이다.

*메타모델 : 개별 모델의 예측된 데이터세트를 다시 기반으로 하여 학습하고 예측하는 모델*


## 1. 데이터 로딩

* 이번 실습에서는 사이킷런에 내장된 위스콘신 암 데이터 세트를 사용하겠습니다.

In [4]:
import numpy as np

# 사용할 개별 알고리즘들
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# DataSet 로딩
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
cancer_ = load_breast_cancer()

X_data = cancer_.data
y_label = cancer_.target

X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, test_size = 0.2, random_state = 0)

## 2. 개별 모델 학습/예측/평가
* 개별 모델 Classifier 생성


In [13]:
knn_clf = KNeighborsClassifier()
rf_clf = RandomForestClassifier()
ada_clf = AdaBoostClassifier()
dt_clf = DecisionTreeClassifier()

 # 최종적으로 개별모델들의 예측 결과를 사용해 다시 예측할 모델
lr_final = LogisticRegression(C=11)

* 개별 Classifier 학습

In [15]:
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier()

* 개별 Classifier 예측

In [18]:
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

* 개별 Classifier 정확도 확인

In [19]:
print("KNN 정확도 : {0:4f}".format(accuracy_score(y_test, knn_pred)))
print("Random Forest Tree 정확도 : {0:4f}".format(accuracy_score(y_test, rf_pred)))
print("Decision Tree 정확도 : {0:4f}".format(accuracy_score(y_test, dt_pred)))
print("AdaBoost 정확도 : {0:4f}".format(accuracy_score(y_test, ada_pred)))

KNN 정확도 : 0.938596
Random Forest Tree 정확도 : 0.973684
Decision Tree 정확도 : 0.912281
AdaBoost 정확도 : 0.956140


* 개별 모델의 예측 결과를 스태킹 형태로 재생성

In [20]:
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(pred.shape)

(4, 114)


* 스태킹 형태는 각 결과가 행이 아닌 열로 배치되어있어야 한다. 즉. transpose로 형태를 뒤집어줘야 한다.

In [21]:
# transpose를 이용해 행과 열의 위치 교환. 컬럼 레벨로 각 알고리즘의 예측 결과를 피처로 만듦. 
pred = np.transpose(pred)
print(pred.shape)

(114, 4)


## 3. 메타모델 학습/예측/평가

In [24]:
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)

print("최종 모델의 정확도 :{0:4f}".format(accuracy_score(y_test,final)))

최종 모델의 정확도 :0.982456


## 정확도가 개선되었지만 <br>스태킹모델에서 y_test를 학습데이터로 사용했기 때문에 <br>과적합이 됐을 확률이 크다. <br>이를 개선하기 위해 CV세트 기반 스태킹 모델이 효과가 있을지 검증해 본다.

## 4. CV 교차검증 기반 스태킹 *******

아직 class가 만들어지지 않아서 수작업으로 해야되고 다소 복잡한 KFold와 Stacking을 결합했기 때문에 복잡한 모델

In [27]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수. 
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=0)
    
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  
        
        #아직까진 KFold 내부에서 학습수행 !!
        model.fit(X_tr , y_tr)
        
        
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        # 이거는 KFOld의 검증데이터를 예측하는거지 실제 테스트용 데이터가 아님 !!!!!
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1) #2차원으로 변경
        
        
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean


##각각의 개별모델들이 반환하는 결과값

In [28]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test,  7)    
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7)

C:\Users\kimde\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


KNeighborsClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
RandomForestClassifier  model 시작 
	 폴드 세트:  0  시작 


C:\Users\kimde\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 


C:\Users\kimde\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(
C:\Users\kimde\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


DecisionTreeClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
AdaBoostClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 


In [29]:
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)
print('원본 학습 피처 데이터 Shape:',X_train.shape, '원본 테스트 피처 Shape:',X_test.shape)
print('스태킹 학습 피처 데이터 Shape:', Stack_final_X_train.shape,
      '스태킹 테스트 피처 데이터 Shape:',Stack_final_X_test.shape)

원본 학습 피처 데이터 Shape: (455, 30) 원본 테스트 피처 Shape: (114, 30)
스태킹 학습 피처 데이터 Shape: (455, 4) 스태킹 테스트 피처 데이터 Shape: (114, 4)


In [30]:
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)


#원본 테스트 데이터의 레이블과 비교
print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_test, stack_final)))

최종 메타 모델의 예측 정확도: 0.9825
